# Graph isomorphism

Ising formulation here: https://arxiv.org/abs/1302.5843 (Lucas 2014)

Lets assume that we have two directed graphs $G_1 = (V_1, E_1)$ and $G_2 = (V_2, E_2)$. They are isomorphic if we can find bijection from $V_1$ to $V_2$ so that structure of mapped $V_1$ is identical to $V_2$. Obviously $n=|V_1|=|V_2|$. 

One obvious solution is binary variable vector having $|V||V|$ elements. Each element indicate a mapping from one vertex to the other \citep{lucas_ising_2014}. To achieve the results intended, following constraints are build to QUBO:
1. every vertex $v_1 \in G_1$ must be mapped to some vertex $v_3 \in G_2$ (penalty $p$ for every couple of vertices which are indicated to be mappings from the same source)  
2. every vertex $v_2 \in G_2$ must be mapped from some vertex $v_1 \in G_1$ (penalty $p$ for every couple of vertices which are indicated to be mapped to the same target) 
3. for every $e_1 \in E_1$  there should be counterpart $e_2 \in E_2$ so that with mapping $e_1$ starting and ending vertex there are similar $e_2$. Each succesfull mapping brings gain of $-1$. 

If graphs are isomorphic, lowest energy level is $|E|$. This comes from third constraint of the QUBO. Binary variable vector of this energy level shows correct mapping.  

Main disadvantage of this algorithm is fast growing elements in binary variable vector when number of vertices increases (size of vector is $|V|^2$). Size of QUBO grows even faster, as size of the QUBO is $|V|^4$.

In [2]:
import numpy as np
import time
import dimod
from dwave.system import DWaveSampler, EmbeddingComposite, LeapHybridSampler
from dwave.samplers import SimulatedAnnealingSampler
import dwave.inspector
import networkx as nx
from networkx.classes.function import path_weight
import random

## QUBO function

In [3]:
def create_qubo(E1,E2,vertices,p):
    Q = np.zeros((vertices*vertices, vertices*vertices))
    
    # Constraint 1: penalty if several mappings from same source
    for i in range(vertices): 
        for j in range(vertices): 
            for k in range(j+1,vertices): 
                Q[i*vertices+j,i*vertices+k]=p 

    # Constaint 2: penalty if several mappings to same target
    for i in range(vertices): 
        for j in range(vertices): 
            for k in range(j+1,vertices): 
                Q[i+vertices*j,i+vertices*k]=p 
                
    # Constraint 3: -1 for each succesfully mapped edge
    for e1 in E1: 
        for e2 in E2: 
            Q[e1[0]*vertices+e2[0], e1[1]*vertices+e2[1]] -= 1 
            
    return Q

## Helper functions

In [29]:
def result_info(sample, e):
    print('Lowest energy should be:',-e)
    print('Lowest energy was:',int(sample.energy))
    if -e==int(sample.energy):
        print('Graphs are isomorphic')
    else:
        print('Graphs are NOT isomorphic')
    m = ''
    res = {}
    for k,v in sample.sample.items():
        if v==1:
            m += str(k)+', '
            res[k[0]]=k[1]
    print('Mapping: '+m)
    return res
    
def solve_gi(solver, num_reads, E1, E2, vertices, p, qpu_info=False, timing_info=False):
    Q = create_qubo(E1,E2,vertices,p)

    labels = {}
    for i in range(vertices):
        for j in range(vertices):
            labels[i*vertices+j] = (i,j)
        
    bqm = dimod.BinaryQuadraticModel(Q, 'BINARY')
    bqm = bqm.relabel_variables(labels, inplace=False)
    t1 = time.time()
    sampleset = solver.sample(bqm, num_reads=num_reads).aggregate()
    ttime = (time.time()-t1)*1000
    res = result_info(sampleset.first, len(E1))
    if qpu_info:
        print('\nTotal time used (ms): {:.3f}'.format(ttime))
        print('Number of logical qubits:',Q.shape[0])
        print('Number of couplers:', len(bqm.quadratic))
        qpu_time = sampleset.info['timing']['qpu_access_time'] / 1000
        qubits = sum(len(x) for x in sampleset.info['embedding_context']['embedding'].values())
        print('QPU time used (ms): {:.1f}'.format(qpu_time))
        print('Physical qubits used: {}\n'.format(qubits))
    if timing_info:
        print('\nTotal time used (ms): {:.3f}'.format(ttime))

    return res

## Simple graphs

In [5]:
vertices = 5
E1 = np.array([(0, 2), (2, 1), (1, 3), (3, 2), (0, 1), (3, 4), (2, 4)])
E3 = np.array([(0, 2), (2, 1), (1, 3), (3, 2), (0, 1), (3, 4), (2, 0)]) # last number difference

perm={0:2, 1:1, 2:3, 3:4, 4:0}
E2 = np.array(E1, copy=True) 
for i in range(len(E2)):
    E2[i]=(perm[E2[i][0]],perm[E2[i][1]])
    
p = len(E2)
print('Penalty:',p)

Penalty: 7


### Heuristic solver

In [6]:
print('Identical graphs')
res = solve_gi(SimulatedAnnealingSampler(), 1000, E1, E1, vertices, p)

Identical graphs
Lowest energy should be: -7
Lowest energy was: -7
Graphs are isomorphic
Mapping: (0, 0), (1, 1), (2, 2), (3, 3), (4, 4), 


In [7]:
print('Differing graphs')
res = solve_gi(SimulatedAnnealingSampler(), 1000, E1, E3, vertices, p)

Differing graphs
Lowest energy should be: -7
Lowest energy was: -6
Graphs are NOT isomorphic
Mapping: (0, 0), (1, 1), (2, 2), (3, 3), (4, 4), 


In [8]:
print('Identical but nodes permuted')
res = solve_gi(SimulatedAnnealingSampler(), 1000, E1, E2, vertices, p)

Identical but nodes permuted
Lowest energy should be: -7
Lowest energy was: -7
Graphs are isomorphic
Mapping: (0, 2), (1, 1), (2, 3), (3, 4), (4, 0), 


### Quantum solver

In [9]:
machine = DWaveSampler(solver={'chip_id': 'Advantage_system4.1'})
print('Chip:', machine.properties['chip_id'])
print('Qubits:', machine.properties['num_qubits'])

Chip: Advantage_system4.1
Qubits: 5760


In [10]:
print('Identical graphs')
res = solve_gi(EmbeddingComposite(machine), 200, E1, E1, vertices, p, qpu_info=True)

Identical graphs
Lowest energy should be: -7
Lowest energy was: -7
Graphs are isomorphic
Mapping: (0, 0), (1, 1), (2, 2), (3, 3), (4, 4), 

Number of logical qubits: 25
Number of couplers: 149
QPU time used (ms): 46.3
Physical qubits used: 65



In [11]:
print('Differing graphs')
res = solve_gi(EmbeddingComposite(machine), 200, E1, E3, vertices, p, qpu_info=True)

Differing graphs
Lowest energy should be: -7
Lowest energy was: -6
Graphs are NOT isomorphic
Mapping: (0, 0), (1, 1), (2, 2), (3, 3), (4, 4), 

Number of logical qubits: 25
Number of couplers: 149
QPU time used (ms): 37.3
Physical qubits used: 73



In [12]:
print('Identical but nodes permuted')
res = solve_gi(EmbeddingComposite(machine), 200, E1, E2, vertices, p, qpu_info=True)

Identical but nodes permuted
Lowest energy should be: -7
Lowest energy was: -7
Graphs are isomorphic
Mapping: (0, 2), (1, 1), (2, 3), (3, 4), (4, 0), 

Number of logical qubits: 25
Number of couplers: 149
QPU time used (ms): 46.6
Physical qubits used: 70



## Larger graph

In [13]:
seed = 42
vertices2 = 10
random.seed(seed)
G = nx.gnp_random_graph(vertices2, 0.30, seed, directed=True)
E4 = [] 
for e in G.edges(data=True):
    E4.append((e[0],e[1]))
print('Number of edges:',len(E4))
print('Number of vertices:',vertices2)
p = len(E4)
print('Penalty:',p)

Number of edges: 34
Number of vertices: 10
Penalty: 34


In [14]:
mapping = dict(zip(G.nodes(), sorted(G.nodes(), key=lambda k: random.random())))
print(mapping)
G2 = nx.relabel_nodes(G, mapping)
E5 = []
for e in G2.edges(data=True):
    E5.append((e[0],e[1]))

{0: 1, 1: 9, 2: 7, 3: 3, 4: 2, 5: 8, 6: 0, 7: 5, 8: 4, 9: 6}


In [15]:
G3 = nx.gnp_random_graph(vertices2, 0.30, seed+1, directed=True)
E6 = [] 
for e in G3.edges(data=True):
    E6.append((e[0],e[1]))

### Local heuristic solver

In [30]:
print('Identical graphs')
res = solve_gi(SimulatedAnnealingSampler(), 1000, E4, E4, vertices2, p, timing_info=True)

Identical graphs
Lowest energy should be: -34
Lowest energy was: -34
Graphs are isomorphic
Mapping: (0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9), 

Total time used (ms): 3115.777


In [31]:
print('Similar graphs')
res = solve_gi(SimulatedAnnealingSampler(), 1000, E4, E5, vertices2, p, timing_info=True)

Similar graphs
Lowest energy should be: -34
Lowest energy was: -34
Graphs are isomorphic
Mapping: (0, 1), (1, 9), (2, 7), (3, 3), (4, 2), (5, 8), (6, 0), (7, 5), (8, 4), (9, 6), 

Total time used (ms): 1922.795


In [32]:
print('Different graphs')
res = solve_gi(SimulatedAnnealingSampler(), 1000, E4, E6, vertices2, p, timing_info=True)

Different graphs
Lowest energy should be: -34
Lowest energy was: -18
Graphs are NOT isomorphic
Mapping: (0, 8), (1, 7), (2, 5), (3, 9), (4, 2), (5, 3), (6, 4), (7, 0), (8, 1), (9, 6), 

Total time used (ms): 1819.575


### Qauntum solver

In [19]:
machine = DWaveSampler(solver={'chip_id': 'Advantage_system4.1'})
print('Chip:', machine.properties['chip_id'])
print('Qubits:', machine.properties['num_qubits'])

Chip: Advantage_system4.1
Qubits: 5760


In [21]:
print('Identical graphs')
res = solve_gi(EmbeddingComposite(machine), 1000, E4, E4, vertices2, p, qpu_info=True)

Identical graphs
Lowest energy should be: -34
Lowest energy was: -12
Graphs are NOT isomorphic
Mapping: (0, 7), (1, 9), (3, 6), (4, 3), (5, 0), (7, 2), (8, 8), (9, 5), 

Number of logical qubits: 100
Number of couplers: 1928
QPU time used (ms): 227.8
Physical qubits used: 1246



In [24]:
print('Similar graphs')
res = solve_gi(EmbeddingComposite(machine), 1000, E4, E5, vertices2, p, qpu_info=True)

Similar graphs
Lowest energy should be: -34
Lowest energy was: -12
Graphs are NOT isomorphic
Mapping: (0, 7), (1, 5), (2, 1), (3, 9), (4, 0), (5, 8), (6, 2), (8, 4), 
Total time used (ms): 50800.370


Number of logical qubits: 100
Number of couplers: 1928
QPU time used (ms): 256.7
Physical qubits used: 1165



In [26]:
print('Different graphs')
res = solve_gi(EmbeddingComposite(machine), 1000, E4, E6, vertices2, p, qpu_info=True)

Different graphs
Lowest energy should be: -34
Lowest energy was: -10
Graphs are NOT isomorphic
Mapping: (0, 6), (1, 3), (2, 8), (3, 9), (4, 2), (5, 7), (6, 4), (7, 0), (8, 1), 

Total time used (ms): 52746.454
Number of logical qubits: 100
Number of couplers: 1634
QPU time used (ms): 240.8
Physical qubits used: 1109



## Testing procedure

In [92]:
for seed in range(20):
    vertices = 10
    random.seed(seed)
    G1 = nx.gnp_random_graph(vertices2, 0.30, seed, directed=True)
    E1 = [] 
    for e in G1.edges(data=True):
        E1.append((e[0],e[1]))
    p = len(E1)
    
    mapping = dict(zip(G.nodes(), sorted(G.nodes(), key=lambda k: random.random())))
    G2 = nx.relabel_nodes(G1, mapping)
    E2 = []
    for e in G2.edges(data=True):
        E2.append((e[0],e[1]))
        
    res = solve_gi(SimulatedAnnealingSampler(), 1000, E1, E2, vertices, p)
    if res==mapping:
        print('Result ok\n')
    else:
        print('ERROR\n')

Lowest energy should be: -17
Lowest energy was: -17
Graphs are isomorphic
Mapping: (0, 3), (1, 7), (2, 5), (3, 2), (4, 8), (5, 4), (6, 9), (7, 1), (8, 6), (9, 0), 
Result ok

Lowest energy should be: -26
Lowest energy was: -26
Graphs are isomorphic
Mapping: (0, 9), (1, 8), (2, 0), (3, 3), (4, 5), (5, 4), (6, 6), (7, 2), (8, 7), (9, 1), 
Result ok

Lowest energy should be: -19
Lowest energy was: -19
Graphs are isomorphic
Mapping: (0, 2), (1, 3), (2, 7), (3, 8), (4, 9), (5, 6), (6, 5), (7, 4), (8, 1), (9, 0), 
Result ok

Lowest energy should be: -23
Lowest energy was: -23
Graphs are isomorphic
Mapping: (0, 6), (1, 5), (2, 9), (3, 0), (4, 8), (5, 2), (6, 1), (7, 3), (8, 4), (9, 7), 
Result ok

Lowest energy should be: -30
Lowest energy was: -30
Graphs are isomorphic
Mapping: (0, 4), (1, 1), (2, 3), (3, 9), (4, 0), (5, 2), (6, 5), (7, 8), (8, 7), (9, 6), 
Result ok

Lowest energy should be: -29
Lowest energy was: -29
Graphs are isomorphic
Mapping: (0, 6), (1, 7), (2, 0), (3, 9), (4, 4), (5